In [31]:
import time
import requests
from bs4 import BeautifulSoup
import urllib
import os

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager

browser = webdriver.Chrome(ChromeDriverManager().install())
# driver = webdriver.Chrome("C:Downloads/chromedriver")
# browser = webdriver.Chrome()

URL = "https://www.nba.com/players"
browser.get(URL)
r = requests.get(URL)
soup = BeautifulSoup(r.content, 'html5lib')
# time.sleep(1)

elem = browser.find_element_by_tag_name("body")

no_of_pagedowns = 120

while no_of_pagedowns:
    elem.send_keys(Keys.PAGE_DOWN)
    # time.sleep(0.2)
    no_of_pagedowns-=1

# after scrolling, I make a new folder
src = "./originalPics"
os.mkdir(src)

digit = 1
# nbaImages = soup.findAll('img', attrs = {'class':'lazyloaded'})
# nbaImages = soup.findAll('img', attrs = {'class':'lazyload'})

post_elems = browser.find_elements_by_class_name("lazyload")
post_elems = browser.find_elements_by_class_name("lazyloaded")

count = 0

for post in post_elems:
    # get the image link
    link = post.get_attribute("data-src")
    if link[:2] == "//":
        link = "https://" + link[2:]

    if count == 250:
        break
    count += 1

    # name of the file
    filename = "nba" + str(count)

    # actually write the image in the folder
    mergeFile = os.path.join(src, filename + ".jpeg")
    output = open(mergeFile, 'wb')
    output.write(urllib.request.urlopen(link).read())
    output.close()

# for img in nbaImages:
#     link = img.get('data-src')

#     if link[:2] == "//":
#         link = "https://" + link[2:]
#     print(link) 

#     if digit == 351:
#         break

#     strVal = str(digit)
#     filename = "nba" + strVal
#     digit += 1

#     mergeFile = os.path.join(src, filename + ".jpeg")
#     output = open(mergeFile, 'wb')
#     output.write(urllib.request.urlopen(link).read())
#     output.close()

# post_elems = browser.find_elements_by_class_name("lazyload")
# post_elems = browser.find_elements_by_class_name("lazyloaded")
# print("aiazng")
# count = 1

# for post in post_elems:
#     print(str(count))
#     count += 1
#     print (post.text)
# print("whats gooddd")
#web scraped NBA photos
# import requests
# from bs4 import BeautifulSoup
# import urllib
# import os

# URL = "https://www.nba.com/players"
# r = requests.get(URL)

# origin = "GAN"
# src = "./originalPics"

# soup = BeautifulSoup(r.content, 'html5lib')
# print(soup.prettify()) 


# digit = 1
# nbaImages = soup.findAll('img', attrs = {'class':'lazyloaded'})

# os.mkdir(src)

# for img in nbaImages:
#     link = img.get('data-src')

#     if link[:2] == "//":
#         link = "https://" + link[2:]
#     print(link) 

#     if digit == 351:
#         break

#     strVal = str(digit)
#     filename = "nba" + strVal
#     digit += 1

#     mergeFile = os.path.join(src, filename + ".jpeg")
#     output = open(mergeFile, 'wb')
#     output.write(urllib.request.urlopen(link).read())
#     output.close()

[WDM] - Current google-chrome version is 85.0.4183
INFO:WDM:Current google-chrome version is 85.0.4183
[WDM] - Get LATEST driver version for 85.0.4183
INFO:WDM:Get LATEST driver version for 85.0.4183
[WDM] - Driver [/Users/kevinmo/.wdm/drivers/chromedriver/mac64/85.0.4183.87/chromedriver] found in cache
INFO:WDM:Driver [/Users/kevinmo/.wdm/drivers/chromedriver/mac64/85.0.4183.87/chromedriver] found in cache
 
aiazng


In [32]:
# this is resizing the images
# %matplotlib inline
# import os
# import PIL
# from PIL import Image

import os
import cv2

# put images in folder
src = "./originalPics"
dst = "./resized"

os.mkdir(dst)

count = 1

for pic in os.listdir(src):
    img = cv2.imread(os.path.join(src,pic))
    img = cv2.resize(img,(128, 128))
    cv2.imwrite(os.path.join(dst,pic), img)



In [33]:
from PIL import Image
import os
src = "resized"
dst = "./resized_black/"

os.mkdir(dst)

for pic in os.listdir(src):
    png = Image.open(os.path.join(src,pic))
    # print each
    if png.mode == 'RGBA':
        png.load() # required for png.split()
        background = Image.new("RGB", png.size, (0,0,0))
        background.paste(png, mask=png.split()[3]) # 3 is the alpha channel
        background.save(os.path.join(dst,pic.split('.')[0] + '.jpeg'), 'JPEG') # changed from .jpg
    else:
        png.convert('RGB')
        png.save(os.path.join(dst,pic.split('.')[0] + '.jpeg'), 'JPEG')
print("debug")

debug


In [34]:
!pip install tensorflow==1.15

import os
import tensorflow as tf
# import tensorflow.compat.v1 as tf
import numpy as np
import cv2
import random
import scipy.misc
import tf_slim as slim
# from python_utils import *
from utils import *

# slim = tf.contrib.slim
print(tf.__version__)

# variables needed
HEIGHT, WIDTH, CHANNEL = 128, 128, 3
BATCH_SIZE = 64
EPOCH = 1000
version = 'newFaces'
newFace_path = './' + version

print("hi")

1.15.0
yuh
hi


In [35]:
### LEAKY RELU - if gradient is too small, we add leak to it to keep it going
def lrelu(x, n, leak=0.2): 
    return tf.maximum(x, x * leak, name=n)
print("hello")

hello


In [36]:
### PROCESS THE DATA, get two variables (shown at end)

def process_data():   
    current_dir = os.getcwd()
    face_dir = os.path.join(current_dir, 'resized_black')
    images = []

    # put images in resized black folder
    for each in os.listdir(face_dir):
        images.append(os.path.join(face_dir,each))

    all_images = tf.convert_to_tensor(images, dtype = tf.string)
    
    images_queue = tf.train.slice_input_producer(
                                        [all_images])
                                        
    content = tf.read_file(images_queue[0])
    image = tf.image.decode_jpeg(content, channels = CHANNEL)
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_brightness(image, max_delta = 0.1)
    image = tf.image.random_contrast(image, lower = 0.9, upper = 1.1)
    # print image.get_shape()
    size = [HEIGHT, WIDTH]
    image = tf.image.resize_images(image, size)
    image.set_shape([HEIGHT,WIDTH,CHANNEL])
    # image = image + noise
    
    # get rid of tf warning
    image = tf.cast(image, tf.float32)
    image = image / 255.0
    
    iamges_batch = tf.train.shuffle_batch(
                                    [image], batch_size = BATCH_SIZE,
                                    num_threads = 4, capacity = 200 + 3* BATCH_SIZE,
                                    min_after_dequeue = 200)
    num_images = len(images)

# returns batch of photos and quantity of images
    return iamges_batch, num_images


In [37]:
# generator network

tf.reset_default_graph()
def generator(input, random_dim, is_train, reuse=False):
    # used for network
    c4, c8, c16, c32, c64 = 512, 256, 128, 64, 32 
    s4 = 4
    output_dim = CHANNEL  
    with tf.variable_scope('gen') as scope:
        if reuse:
            scope.reuse_variables()

        # build convolution and the layers of the network
        #1
        w1 = tf.get_variable('w1', shape=[random_dim, s4 * s4 * c4], dtype=tf.float32,
                             initializer=tf.truncated_normal_initializer(stddev=0.02))
        b1 = tf.get_variable('b1', shape=[c4 * s4 * s4], dtype=tf.float32,
                             initializer=tf.constant_initializer(0.0))
        flat_conv1 = tf.add(tf.matmul(input, w1), b1, name='flat_conv1')
        conv1 = tf.reshape(flat_conv1, shape=[-1, s4, s4, c4], name='conv1')
        bn1 = tf.contrib.layers.batch_norm(conv1, is_training=is_train, epsilon=1e-5, decay = 0.9,  updates_collections=None, scope='bn1')
        act1 = tf.nn.relu(bn1, name='act1')
        #2
        conv2 = tf.layers.conv2d_transpose(act1, c8, kernel_size=[5, 5], strides=[2, 2], padding="SAME",
                                           kernel_initializer=tf.truncated_normal_initializer(stddev=0.02),
                                           name='conv2')
        bn2 = tf.contrib.layers.batch_norm(conv2, is_training=is_train, epsilon=1e-5, decay = 0.9,  updates_collections=None, scope='bn2')
        act2 = tf.nn.relu(bn2, name='act2')
        #3
        conv3 = tf.layers.conv2d_transpose(act2, c16, kernel_size=[5, 5], strides=[2, 2], padding="SAME",
                                           kernel_initializer=tf.truncated_normal_initializer(stddev=0.02),
                                           name='conv3')
        bn3 = tf.contrib.layers.batch_norm(conv3, is_training=is_train, epsilon=1e-5, decay = 0.9,  updates_collections=None, scope='bn3')
        act3 = tf.nn.relu(bn3, name='act3')
        #4
        conv4 = tf.layers.conv2d_transpose(act3, c32, kernel_size=[5, 5], strides=[2, 2], padding="SAME",
                                           kernel_initializer=tf.truncated_normal_initializer(stddev=0.02),
                                           name='conv4')
        bn4 = tf.contrib.layers.batch_norm(conv4, is_training=is_train, epsilon=1e-5, decay = 0.9,  updates_collections=None, scope='bn4')
        act4 = tf.nn.relu(bn4, name='act4')
        #5
        conv5 = tf.layers.conv2d_transpose(act4, c64, kernel_size=[5, 5], strides=[2, 2], padding="SAME",
                                           kernel_initializer=tf.truncated_normal_initializer(stddev=0.02),
                                           name='conv5')
        bn5 = tf.contrib.layers.batch_norm(conv5, is_training=is_train, epsilon=1e-5, decay = 0.9,  updates_collections=None, scope='bn5')
        act5 = tf.nn.relu(bn5, name='act5')
        
        #6
        conv6 = tf.layers.conv2d_transpose(act5, output_dim, kernel_size=[5, 5], strides=[2, 2], padding="SAME",
                                           kernel_initializer=tf.truncated_normal_initializer(stddev=0.02),
                                           name='conv6')
        # bn6 = tf.contrib.layers.batch_norm(conv6, is_training=is_train, epsilon=1e-5, decay = 0.9,  updates_collections=None, scope='bn6')
        act6 = tf.nn.tanh(conv6, name='act6')
        return act6
print("hello")

hello


In [38]:
#### discriminator network
def discriminator(input, is_train, reuse=False):
    c2, c4, c8, c16 = 64, 128, 256, 512  
    with tf.variable_scope('dis') as scope:
        if reuse:
            scope.reuse_variables()

        # layers of discriminator network
        #1
        conv1 = tf.layers.conv2d(input, c2, kernel_size=[5, 5], strides=[2, 2], padding="SAME",
                                 kernel_initializer=tf.truncated_normal_initializer(stddev=0.02),
                                 name='conv1')
        bn1 = tf.contrib.layers.batch_norm(conv1, is_training = is_train, epsilon=1e-5, decay = 0.9,  updates_collections=None, scope = 'bn1')
        act1 = lrelu(conv1, n='act1')
        #2
        conv2 = tf.layers.conv2d(act1, c4, kernel_size=[5, 5], strides=[2, 2], padding="SAME",
                                 kernel_initializer=tf.truncated_normal_initializer(stddev=0.02),
                                 name='conv2')
        bn2 = tf.contrib.layers.batch_norm(conv2, is_training=is_train, epsilon=1e-5, decay = 0.9,  updates_collections=None, scope='bn2')
        act2 = lrelu(bn2, n='act2')
        #3
        conv3 = tf.layers.conv2d(act2, c8, kernel_size=[5, 5], strides=[2, 2], padding="SAME",
                                 kernel_initializer=tf.truncated_normal_initializer(stddev=0.02),
                                 name='conv3')
        bn3 = tf.contrib.layers.batch_norm(conv3, is_training=is_train, epsilon=1e-5, decay = 0.9,  updates_collections=None, scope='bn3')
        act3 = lrelu(bn3, n='act3')
         #4
        conv4 = tf.layers.conv2d(act3, c16, kernel_size=[5, 5], strides=[2, 2], padding="SAME",
                                 kernel_initializer=tf.truncated_normal_initializer(stddev=0.02),
                                 name='conv4')
        bn4 = tf.contrib.layers.batch_norm(conv4, is_training=is_train, epsilon=1e-5, decay = 0.9,  updates_collections=None, scope='bn4')
        act4 = lrelu(bn4, n='act4')
       
        # start from 4th layer
        dim = int(np.prod(act4.get_shape()[1:]))
        fc1 = tf.reshape(act4, shape=[-1, dim], name='fc1')
      
        
        w2 = tf.get_variable('w2', shape=[fc1.shape[-1], 1], dtype=tf.float32,
                             initializer=tf.truncated_normal_initializer(stddev=0.02))
        b2 = tf.get_variable('b2', shape=[1], dtype=tf.float32,
                             initializer=tf.constant_initializer(0.0))

        # sigmoid elimination
        logits = tf.add(tf.matmul(fc1, w2), b2, name='logits')
        acted_out = tf.nn.sigmoid(logits)
        return logits
print("hello")


hello


In [39]:
###### THIS IS TRAINING THE MODEL

def train():
    random_dim = 100
    
    with tf.variable_scope('input'):
        tf.compat.v1.disable_eager_execution()
        # placeholders
        real_image = tf.placeholder(tf.float32, shape = [None, HEIGHT, WIDTH, CHANNEL], name='real_image')
        random_input = tf.placeholder(tf.float32, shape=[None, random_dim], name='rand_input')
        is_train = tf.placeholder(tf.bool, name='is_train')
    
    # gets fake/real images and result to fully establish a working
    # generator and discriminator interaction
    fake_image = generator(random_input, random_dim, is_train)
    
    real_result = discriminator(real_image, is_train)
    fake_result = discriminator(fake_image, is_train, reuse=True)
    
    # optimization
    d_loss = tf.reduce_mean(fake_result) - tf.reduce_mean(real_result)  
    g_loss = -tf.reduce_mean(fake_result) 
            

    t_vars = tf.trainable_variables()
    d_vars = [var for var in t_vars if 'dis' in var.name]
    g_vars = [var for var in t_vars if 'gen' in var.name]
    trainer_d = tf.train.RMSPropOptimizer(learning_rate=2e-4).minimize(d_loss, var_list=d_vars)
    trainer_g = tf.train.RMSPropOptimizer(learning_rate=2e-4).minimize(g_loss, var_list=g_vars)
    d_clip = [v.assign(tf.clip_by_value(v, -0.01, 0.01)) for v in d_vars]

    
    batch_size = BATCH_SIZE
    image_batch, samples_num = process_data()
    
    batch_num = int(samples_num / batch_size)
    total_batch = 0
    sess = tf.Session()
    saver = tf.train.Saver()
    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())
    save_path = saver.save(sess, "/tmp/model.ckpt")
    ckpt = tf.train.latest_checkpoint('./model/' + version)
    saver.restore(sess, save_path)
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)

    # show quantity statistics
    print('total training sample num:%d' % samples_num)
    print('batch size: %d, batch num per epoch: %d, epoch num: %d' % (batch_size, batch_num, EPOCH))
    print('start training...')
    # ACTUAL TRAINING STARTS
    for i in range(EPOCH):
        print("Running epoch {}/{}...".format(i, EPOCH))
        for j in range(batch_num):
            print(j)
            d_iters = 5
            g_iters = 1

            train_noise = np.random.uniform(-1.0, 1.0, size=[batch_size, random_dim]).astype(np.float32)
            for k in range(d_iters):
                print(k)
                train_image = sess.run(image_batch)
                sess.run(d_clip)
                
                # Discriminator update
                _, dLoss = sess.run([trainer_d, d_loss],
                                    feed_dict={random_input: train_noise, real_image: train_image, is_train: True})
                print("dLoss is " + str(dLoss))

            # Generator update
            for k in range(g_iters):
                _, gLoss = sess.run([trainer_g, g_loss],
                                    feed_dict={random_input: train_noise, is_train: True})

            print ('train:[%d/%d],d_loss:%f,g_loss:%f' % (i, j, dLoss, gLoss))
            
        # save every 500 epoch
        if i%500 == 0:
            if not os.path.exists('./model/' + version):
                os.makedirs('./model/' + version)
            saver.save(sess, './model/' +version + '/' + str(i))  

        # image save every one epoch
        if i%1 == 0:
            if not os.path.exists(newFace_path):
                os.makedirs(newFace_path)
            sample_noise = np.random.uniform(-1.0, 1.0, size=[batch_size, random_dim]).astype(np.float32)
            imgtest = sess.run(fake_image, feed_dict={random_input: sample_noise, is_train: False})
            # imgtest = imgtest * 255.0
            # imgtest.astype(np.uint8)
            save_images(imgtest, [8,8] ,newFace_path + '/epoch' + str(i) + '.jpg')
            
            print('train:[%d],d_loss:%f,g_loss:%f' % (i, dLoss, gLoss))
    coord.request_stop()
    coord.join(threads)
print("helloooo")

hellofffkevin


In [30]:
###RUNNNING THE MODEL


train()



ValueError: Trying to share variable dis/w2, but specified shape (32768, 1) and found shape (104448, 1).